# 1. 라이브러리 및 API key 설정

In [142]:
import os
import torch
import numpy as np
import datasets
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import roc_auc_score
from openai import OpenAI
from serpapi import GoogleSearch 
from dotenv import load_dotenv
import pandas as pd
import os
import openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langchain_anthropic import ChatAnthropic, AnthropicLLM
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.memory import SimpleMemory
from datetime import datetime
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import json
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

In [140]:


# Load environment variables
load_dotenv()

# Auto-trace LLM calls in-context
client = wrap_openai(openai.Client())


In [141]:
# OpenAI 및 SerpAPI 키 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERP_API_KEY = os.getenv("SERPAPI_API_KEY")

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=OPENAI_API_KEY)

# FAISS 인덱스 저장 경로
FAISS_INDEX_PATH = Path('local/news-please/faiss_index')

# 2. DPR 기반 사내 뉴스 검색 시스템

In [60]:
class DPR():
    def __init__(self):
        """사내 뉴스 데이터베이스를 로드하고 FAISS 인덱스를 활용한 검색을 수행하는 DPR 클래스"""
        self.ds = datasets.load_dataset('sanxing/advfake_news_please')['train']
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.index_dpr()

    @torch.no_grad()
    def index_dpr(self):
        """DPR 기반 뉴스 임베딩을 생성하고 FAISS 인덱스를 구축"""
        from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

        faiss_path = FAISS_INDEX_PATH / 'my_index.faiss'
        if faiss_path.exists():
            print('🔹 FAISS 인덱스 로드 중...')
            self.ds.load_faiss_index('embeddings', str(faiss_path))
            return

        print('🔹 FAISS 인덱스 생성 중...')
        ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(self.device)
        ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

        # 🔹 뉴스 제목을 임베딩 벡터로 변환하여 저장
        ds_with_embeddings = self.ds.map(lambda example: {
            'embeddings': ctx_encoder(**ctx_tokenizer(example["title"], return_tensors="pt", padding=True).to(self.device))[0].cpu().numpy()
        }, batched=True, batch_size=64)

        ds_with_embeddings.add_faiss_index(column='embeddings')

        print('🔹 FAISS 인덱스 저장 중...')
        ds_with_embeddings.save_faiss_index('embeddings', str(faiss_path))

    @torch.no_grad()
    def search(self, query):
        """입력된 쿼리를 기반으로 가장 관련성 높은 뉴스 10개 검색"""
        from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

        q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(self.device)
        q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

        # 🔹 입력 쿼리를 DPR 임베딩으로 변환
        question_embedding = q_encoder(**q_tokenizer(query, return_tensors="pt").to(self.device))[0][0].cpu().numpy()
        
        # 🔹 FAISS 검색 수행
        scores, retrieved_examples = self.ds.get_nearest_examples('embeddings', question_embedding, k=5)

        # 🔹 결과를 딕셔너리 리스트 형태로 변환
        retrieved_examples = [dict(zip(retrieved_examples, t)) for t in zip(*retrieved_examples.values())]

        return scores, retrieved_examples

In [115]:
import torch
import datasets
import faiss
from pathlib import Path
from transformers import (
    DPRContextEncoder, DPRContextEncoderTokenizer,
    DPRQuestionEncoder, DPRQuestionEncoderTokenizer
)

FAISS_INDEX_PATH = Path("faiss_index")  # 저장할 폴더
DATASET_SAVE_PATH = Path("news_dataset")  # 데이터셋 저장 위치
FAISS_FILE = FAISS_INDEX_PATH / "my_index.faiss"

class DPR():
    def __init__(self):
        """사내 뉴스 데이터베이스를 로드하고 FAISS 인덱스를 활용한 검색을 수행하는 DPR 클래스"""
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        # 🔹 저장된 데이터셋이 있으면 로드, 없으면 새로 다운로드
        if DATASET_SAVE_PATH.exists():
            print("🔹 저장된 데이터셋 로드 중...")
            self.ds = datasets.load_from_disk(str(DATASET_SAVE_PATH))
        else:
            print("🔹 새로운 데이터셋 다운로드 중...")
            self.ds = datasets.load_dataset('sanxing/advfake_news_please')['train']
            self.ds.save_to_disk(str(DATASET_SAVE_PATH))  # 저장

        # FAISS 인덱싱
        self.index_dpr()

    @torch.no_grad()
    def index_dpr(self):
        """DPR 기반 뉴스 임베딩을 생성하고 FAISS 인덱스를 구축"""
        if FAISS_FILE.exists():
            print('🔹 FAISS 인덱스 로드 중...')
            self.ds.load_faiss_index('embeddings', str(FAISS_FILE))
            return

        print('🔹 FAISS 인덱스 생성 중...')
        ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(self.device)
        ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

        def embed_batch(examples):
            inputs = ctx_tokenizer(examples["title"], return_tensors="pt", padding=True, truncation=True).to(self.device)
            embeddings = ctx_encoder(**inputs).pooler_output.cpu().numpy()
            return {"embeddings": embeddings}

        ds_with_embeddings = self.ds.map(embed_batch, batched=True, batch_size=64)
        ds_with_embeddings.add_faiss_index(column='embeddings')

        print('🔹 FAISS 인덱스 저장 중...')
        FAISS_INDEX_PATH.mkdir(parents=True, exist_ok=True)
        ds_with_embeddings.save_faiss_index('embeddings', str(FAISS_FILE))

        # 🔹 데이터셋도 함께 저장
        ds_with_embeddings.save_to_disk(str(DATASET_SAVE_PATH))
        self.ds = ds_with_embeddings  # 메모리에 저장

    @torch.no_grad()
    def search(self, query):
        """입력된 쿼리를 기반으로 가장 관련성 높은 뉴스 5개 검색"""
        q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(self.device)
        q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

        # 🔹 입력 쿼리를 DPR 임베딩으로 변환
        inputs = q_tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(self.device)
        question_embedding = q_encoder(**inputs).pooler_output.cpu().numpy()[0]

        # 🔹 FAISS 검색 수행
        scores, retrieved_examples = self.ds.get_nearest_examples('embeddings', question_embedding, k=5)

        # 🔹 결과를 딕셔너리 리스트 형태로 변환
        retrieved_examples = [dict(zip(retrieved_examples, t)) for t in zip(*retrieved_examples.values())]

        return scores, retrieved_examples

In [114]:
pip install torch datasets faiss-cpu transformers

Note: you may need to restart the kernel to use updated packages.


# 3. Google SerpAPI 기반

In [61]:
def search_serpapi(q):
    """Google SerpAPI를 사용하여 실시간 뉴스 검색"""
    params = {
        "api_key": SERP_API_KEY,  # SerpAPI 키
        "engine": "google",  # Google 검색 엔진 사용
        "q": q,  # 검색어
        "location": "Austin, Texas, United States",  # 검색 지역 지정
        "google_domain": "google.com",
        "gl": "us",  # 국가 설정 (미국)
        "hl": "en",  # 언어 설정 (영어)
        "num": "30"  # 검색 결과 최대 30개 가져오기
    }

    search = GoogleSearch(params)  # SerpAPI를 사용한 Google 검색 객체 생성
    results = search.get_dict()  # JSON 형식으로 검색 결과 가져오기
    return results  # 결과 반환

def concat_snippets(organic_results):
    """검색된 기사에서 유의미한 정보를 필터링하고 요약"""
    organic_results = [result for result in organic_results if 'snippet' in result]  # snippet이 있는 기사만 필터링
    organic_results = [result for result in organic_results if 'NBC' not in result['source'] and 'NBC' not in result['title']]  # NBC 뉴스 제외
    organic_results = [result for result in organic_results if 'fact' not in result['link']]  # 'fact'가 포함된 링크 제외 (팩트체크 기사 필터링)
    organic_results = organic_results[:5]  # 상위 5개 기사만 선택

    return '\n'.join([
        f'Title: {result["title"]}\nSource: {result["source"]}, {result["date"] if "date" in result else ""}\nContent: {result["snippet"]}' 
        for result in organic_results
    ])

def get_google_ctx(q):
    """실시간 검색 결과를 LLM 입력에 사용할 형태로 변환"""
    search_results = search_serpapi(q)  # Google 검색 실행
    if 'organic_results' in search_results:
        return concat_snippets(search_results['organic_results'])  # 검색 결과 정리 후 반환
    else:
        return ""

# 4. RAG

In [158]:
def retrieve_relevant_news(news_headline):
    """사내 뉴스 및 Google 검색을 결합하여 관련 뉴스 검색"""
    #dpr = DPR()
    #_, retrieved_news = dpr.search(news_headline)
    retrieved_news=[]
    print(f"retrieved_news:{retrieved_news}")
    google_results = get_google_ctx(news_headline)
    print(google_results)
    
    return retrieved_news + [google_results]

def get_plausibility_score(news_text, retrieved_news):
    """GPT-4o를 이용하여 뉴스의 개연성 점수 예측"""
    input_data = {"news_text":news_text, "retrieved_news" : retrieved_news}
    result = chain_1.invoke(input_data)
    print(result)
    return result['plausibility_score']


def generate_final_score(news_text, retrieved_news):
    """샘플링을 통해 최종 개연성 점수 생성 (0~1 범위)"""
    scores = [get_plausibility_score(news_text, retrieved_news)['plausibility_score'] for _ in range(3)]
    scores = list(map(int, scores))
    final_score = sum(scores) / len(scores)
    return final_score / 10  # 0~1 범위로 정규화

In [161]:
# 의학과학개요 검토 LLM
llm1 = ChatOpenAI(temperature=1,               # 창의성 (0.0 ~ 2.0) 
                 max_tokens=2048,             # 최대 토큰수
                 model_name='gpt-4o',  # 모델명
                )

llm2 = ChatAnthropic(model='claude-3-5-sonnet-20240620',max_tokens=8192)


# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 출력 형식 지시사항을 파싱
format_instructions = output_parser.get_format_instructions()

response_schemas = [
    ResponseSchema(
        name="plausibility_score",
        description="A numerical score between 1 and 10 that represents how plausible the given statement is. "
                    "1 means 'completely false', while 10 means 'fully plausible'."
    ),
    ResponseSchema(
        name="reason",
        description="A textual explanation providing the reasoning behind the assigned plausibility score."
    ),
]

prompt_template_1 = """

    Today is March 26, 2024. You predict the plausibility of a news you haven’t seen.

    News: "{news_text}"

    Retrieved News:
    {retrieved_news}

    Based on the provided information, rate the plausibility of the news on a scale of 1 to 10 (1: completely false, 10: fully plausible). Also, give the proper reason.
    
    """



prompt1 = PromptTemplate(
    template=prompt_template_1,
    input_variables=["news_text","retrieved news"],
    partial_variables={"format_instructions": format_instructions},
)

chain1 = prompt1 | llm2 | output_parser  # 프롬프트, 모델, 출력 파서를 연결




# Test

In [162]:
# 🔹 테스트 데이터 로드 (가상의 뉴스 데이터)
test_news = ["테스트 뉴스 1", "테스트 뉴스 2", "테스트 뉴스 3"]
true_labels = [1, 0, 1]  # 1: Real, 0: Fake

# 🔹 개연성 점수 예측
predicted_scores = [generate_final_score(news, retrieve_relevant_news(news)) for news in test_news]

# 🔹 AUC-ROC 계산
auc_score = roc_auc_score(true_labels, predicted_scores)
print(f"AUC-ROC Score: {auc_score:.4f}")

retrieved_news:[]
Title: 뉴스1
Source: 뉴스1, 
Content: 윤석열 탄핵심판·이재명 선거법 2심 변론종결… · "자백하고 사과" vs "정당성 부각"…'尹 최후진술' 놓고도 시민사회 '쩍' · [탄핵심판의 얼굴들]④국무총리 한덕수…
Title: CONNECT 서비스
Source: 뉴스1, 
Content: 뉴스1, AFP 및 Reuter 뉴스정보 및 사진을 실시간으로 열람할 수 있는 서비스로, 24시간 국내외 속보를 한 눈에 접할 수 있습니다. 매체사,기업, 관공서, 개인 등 이용 ...
Title: 뉴스1 (@News1Kr) / X
Source: x.com, 
Content: 사실 앞에 겸손한 민영 통신 뉴스1입니다. 유튜브 : http://youtube.com/user/news1korea. Translate bio.
Title: 뉴스 테스트
Source: 나무위키, 
Content: 다양한 분야의 뉴스/시사 상식의 습득 수준을 객관적이고 체계적인 지표로 측정할 수 있도록 개발된 뉴스 상식 검정 시험이다. 2017년 3월 25일 시행된 1회 시험에서 최고 ...
Title: Minuteman III test launch showcases readiness of US ...
Source: Space Force (.mil), 4 days ago
Content: Minuteman III test launch showcases readiness of US nuclear force's safe, effective deterrent. Published Feb. 19, 2025; By Staff Sgt. Joshua ...
{'plausibility_score': '3', 'reason': "The news '테스트 뉴스 1' (Test News 1) is very vague and lacks any specific content, making it difficult to assess its plausibility. None of the 

In [159]:
generate_final_score("Trump wins",retrieve_relevant_news("Trump wins") )

retrieved_news:[]
Title: Presidential Election Results Map: Trump Wins
Source: The New York Times, Nov 5, 2024
Content: Donald J. Trump has won the presidency, improving upon his 2020 performance in both red and blue states and capturing enough swing states to reach 270 ...
Title: Wins Come All Day Under President Donald J. Trump
Source: The White House (.gov), Feb 14, 2025
Content: It was another week filled with endless wins for the American people under President Donald J. Trump. Here are only a few of the many ...
Title: Just how big was Donald Trump's election victory?
Source: BBC, Nov 22, 2024
Content: Trump's win over Harris in 2024 appears more comfortable. He won 312 votes in the US electoral college compared with Harris's 226.
Title: Donald Trump wins the 2024 presidential election
Source: NPR, Nov 6, 2024
Content: Becoming just the second president to be defeated and then reelected to a subsequent term, former President Donald Trump defeated Vice ...
Title: The size of Donal

0.9

# Fake News 전처리

In [2]:
import pandas as pd
import numpy as np
# Load Dataset
true_data = pd.read_csv('/Users/yoonjincho/Desktop/FakeNews_kaggle/True.csv')
fake_data = pd.read_csv('/Users/yoonjincho/Desktop/FakeNews_kaggle/Fake.csv')

# Generate labels True/Fake under new Target Column in 'true_data' and 'fake_data'
true_data['label'] = 1
fake_data['label'] = 0

# Merge 'true_data' and 'fake_data', by random mixing into a single df called 'data'
data = pd.concat([true_data, fake_data], ignore_index = True).sample(frac=1).reset_index().drop(columns = ['index'])

# See how the data looks like
print(data.shape)
data.head()

(44898, 5)


,title,text,subject,date,label
0,Appeals court ruling will let some Kansas vote...,(Reuters) - Thousands of Kansas residents who ...,politicsNews,"June 11, 2016",1
1,House Speaker Ryan urges conservative unity in...,WASHINGTON (Reuters) - U.S. House Speaker Paul...,politicsNews,"February 3, 2016",1
2,"Once credited with Trump's success, Bannon qui...",WASHINGTON (Reuters) - When President Donald T...,politicsNews,"August 18, 2017",1
3,"Trump To Gut Coast Guard, Airport Security, A...",Donald Trump s obsession with building a massi...,News,"March 8, 2017",0
4,Trump says Senate Republicans likely to pass h...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"June 28, 2017",1
